# ROI inference sript for Model1
    1. Loading necessary data
    2. Initializing model
    3. Infer whole ROI

## 1. Loading necessary data

In [ ]:
import torch
import torchvision
from torchvision import transforms
from torch import nn as nn
import pickle
import random
import numpy as np
import WHO_Reg_MC4
import openslide
from PIL import Image


torch.cuda.empty_cache()
# without mc
model_file = "models/Model1_seed1.pth"
# with mc
annotation_dict = pickle.load(open("MC_and_ROI_90.p","rb"))

In [2]:
import csv
# File with annotations of test slides
csv_file = "/.csv"
who_grades = dict()
with open(csv_file, newline = '')as file:
    reader = csv.reader(file, delimiter = ',')
    for idx,line in enumerate(reader):
        if idx == 0:
            continue
        who_grades[line[0]] = list()
        who_grades[line[0]].extend([int(line[1]),int(line[2])])

In [3]:
# Flatten structure of annotation dict for Menigeome Erlangen
tmp = dict()

for key in annotation_dict:
    for sub_key in annotation_dict[key]:
        tmp[sub_key] = annotation_dict[key][sub_key]
        

In [4]:
new_annotation = dict()
for key in tmp:
    new_annotation[key] = dict()
    new_annotation[key]['roi'] = tmp[key][1]
    new_annotation[key]['mitotic_count'] = tmp[key][0]
    new_annotation[key]['who_grade'] = who_grades['_'.join(key.split(' ')[0].split('_')[0:2])][0]

annotation_dict = new_annotation

## 2. Initializing Model1 

In [5]:
import torchvision
import torch
from torch import nn, optim
import Model1

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Model1.RegressionModel()
model.load_state_dict(torch.load(model_file))
model.to(device)

RegressionModel(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats

## 3. Infer whole ROI

In [6]:
def regress_batch(images,model,mcs = None):
    device = 'cuda'
    with torch.no_grad():
        if model.training:
            model.eval()
        if mcs == None:
            predictions = model(images.to(device))
        else:
            predictions = model(images.to(device),mcs.to(device))
        return(predictions)

In [7]:
import cv2

def make_active_map(slide):
    downsamples_int = [int(x) for x in slide.level_downsamples]
    if 32 in downsamples_int:
        ds = 32
    elif 16 in downsamples_int:
        ds = 16
    else:
        return
    level = np.where(np.abs(np.array(slide.level_downsamples)-ds)<0.1)[0][0]
    overview = slide.read_region(level=level, location=(0,0), size=slide.level_dimensions[level])
    # convert to grayscale
    gray = cv2.cvtColor(np.array(overview)[:,:,0:3],cv2.COLOR_BGR2GRAY)

    #otsu thresholding
    ret, thresh = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    # dilate
    dil = cv2.dilate(thresh, kernel = np.ones((7,7),np.uint8))

    # erode --> yields map
    activeMap = cv2.erode(dil, kernel = np.ones((7,7),np.uint8))

    return(activeMap)

def find_fullest_roi(activeMap,slide,roi_width,roi_height):
    downsamples_int = [int(x) for x in slide.level_downsamples]
    if 32 in downsamples_int:
        ds = 32
    elif 16 in downsamples_int:
        ds = 16
    else:
        return

    x_length = int(np.ceil(float(roi_width)/ds))
    y_length = int(np.ceil(float(roi_height)/ds))
    kernel = np.ones((x_length,y_length), np.float32)        
    tissue_map = cv2.filter2D(activeMap, -1, kernel, anchor = (0,0), borderType = cv2.BORDER_CONSTANT)
    roi_center = np.unravel_index(tissue_map.argmax(), tissue_map.shape)
    roi_center = [i * ds for i in roi_center]
    return roi_center

In [8]:
import os 
file_list = list()
for root,dirs,files in os.walk("wsis/test_set/",topdown=False):
    for name in files:
        if name.endswith('ndpi') and name in annotation_dict:
            file_list.append(os.path.join(root, name))

In [9]:
from tqdm import tqdm

path_to_wsis = "wsis/test_set/"

transform = transforms.Compose(
    [transforms.ToTensor(),transforms.Normalize([0.8249, 0.5482, 0.7211], [0.0573, 0.0988, 0.0595])])

level = 0
predictions = dict()

model.to('cuda')

for file in tqdm(file_list):
    key = file.split('/')[-1]
    predictions[key] = dict()       
    slide = openslide.open_slide(file)
    if annotation_dict[key]['roi'] == [0, 0, 8284, 6213]:
        print("###### MC == 0!!!######")
        # hier noch aktivste Region bestimmen!!!
        activeMap = make_active_map(slide)
        activeMap = activeMap / activeMap.max()
        top_left = find_fullest_roi(activeMap,slide,8284,6213)
        # top_left = (y,x) due to cv2 convention
        annotation_dict[key]['roi'][0] = top_left[1]
        annotation_dict[key]['roi'][1] = top_left[0]
        annotation_dict[key]['roi'][2] = top_left[1] + annotation_dict[key]['roi'][2]
        annotation_dict[key]['roi'][3] = top_left[0] + annotation_dict[key]['roi'][3]
    predictions[key]['predictions'] = list()
    predictions[key]['label'] = annotation_dict[key]['who_grade']
    for x in range(annotation_dict[key]['roi'][0], annotation_dict[key]['roi'][2],(1024 * (level + 1))):
        for y in range(annotation_dict[key]['roi'][1], annotation_dict[key]['roi'][3],(1024 * (level + 1))):
            img = transform(Image.fromarray(np.array(slide.read_region((x,y),level,(1024,1024)))[:,:,:3]))
            mcs = torch.tensor(annotation_dict[key]['mitotic_count'],dtype = torch.float32).reshape(-1,1)
            # with out mc
            out = regress_batch(img.reshape([1,3,1024,1024]),model)
            predictions[key]['predictions'].append(out.to('cpu').flatten())

   
   
  

  3%|▎         | 4/132 [00:21<11:34,  5.42s/it]

###### MC == 0!!!######


  5%|▌         | 7/132 [00:38<11:32,  5.54s/it]

###### MC == 0!!!######


  7%|▋         | 9/132 [00:50<11:51,  5.78s/it]

...

 90%|█████████ | 119/132 [11:11<01:12,  5.60s/it]

###### MC == 0!!!######


 94%|█████████▍| 124/132 [11:39<00:43,  5.49s/it]

###### MC == 0!!!######


100%|██████████| 132/132 [12:23<00:00,  5.63s/it]


In [10]:
file = ""

pickle.dump(predictions,open(file,"wb"))